In [1]:
import json

with open('./data/json/Primary.json') as json_file:
    data = json.load(json_file)
    
# for item in data:
#     print(item["name"])

In [2]:
data[0]

{'name': 'Acceltra',
 'uniqueName': '/Lotus/Weapons/Tenno/LongGuns/SapientPrimary/SapientPrimaryWeapon',
 'damagePerShot': [26,
  35.200001,
  8.7999992,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'totalDamage': 70,
 'description': 'Using a barrage of rapid-fire plasma rockets, Gauss’ signature weapon lays down a path of destruction. Reloads are faster while sprinting, even more so in Gauss’ hands. For safety, rockets arm after traveling a safe distance.',
 'criticalChance': 0.31999999,
 'criticalMultiplier': 2.8,
 'procChance': 0.060000002,
 'fireRate': 12.000001,
 'masteryReq': 8,
 'productCategory': 'LongGuns',
 'slot': 1,
 'accuracy': 23.529411,
 'omegaAttenuation': 0.60000002,
 'noise': 'Alarming',
 'trigger': 'Auto',
 'magazineSize': 48,
 'reloadTime': 2,
 'multishot': 1,
 'buildPrice': 25000,
 'buildTime': 86400,
 'skipBuildTimePrice': 35,
 'buildQuantity': 1,
 'consumeOnBuild': True,
 'components': [{'uniqueName': '/Lotus/Types/Recipe

## Generating the itemComponents data array and JS file

In [3]:
items = {}
itemNames = []

for item in data[:30]:
    if "components" in item.keys():
        item_components = [{"name": component["name"], "itemCount": component["itemCount"]} for component in item["components"]]
        item_components.append({"name": "Credits", "itemCount": item["buildPrice"]})
        items[item["name"]] = item_components
        itemNames.append(item["name"])

with open("./js/itemComponents.js", "w") as f:
    f.write("var itemComponents = " + json.dumps(items))

with open("./item_names.py", "w") as f:
    f.write("item_names = " + json.dumps(itemNames))

print(itemNames)
items

['Acceltra', 'Amprex', 'Arca Plasmor', 'Argonak', 'Astilla', 'Attica', 'Basmu', 'Battacor', 'Baza', 'Baza Prime', 'Boar', 'Boar Prime', 'Boltor', 'Boltor Prime', 'Braton Prime', 'Braton Vandal', 'Bubonico', 'Burston', 'Burston Prime', 'Buzlok', 'Cernos', 'Cernos Prime', 'Convectrix', 'Corinth', 'Corinth Prime', 'Cortege', 'Corvas']


{'Acceltra': [{'name': 'Blueprint', 'itemCount': 1},
  {'name': 'Hexenon', 'itemCount': 200},
  {'name': 'Nano Spores', 'itemCount': 8000},
  {'name': 'Neurodes', 'itemCount': 4},
  {'name': 'Plastids', 'itemCount': 925},
  {'name': 'Credits', 'itemCount': 25000}],
 'Amprex': [{'name': 'Argon Crystal', 'itemCount': 3},
  {'name': 'Blueprint', 'itemCount': 1},
  {'name': 'Ferrite', 'itemCount': 9000},
  {'name': 'Fieldron', 'itemCount': 8},
  {'name': 'Plastids', 'itemCount': 600},
  {'name': 'Credits', 'itemCount': 25000}],
 'Arca Plasmor': [{'name': 'Blueprint', 'itemCount': 1},
  {'name': 'Control Module', 'itemCount': 25},
  {'name': 'Cryotic', 'itemCount': 925},
  {'name': 'Fieldron', 'itemCount': 5},
  {'name': 'Forma', 'itemCount': 1},
  {'name': 'Credits', 'itemCount': 25000}],
 'Argonak': [{'name': 'Alloy Plate', 'itemCount': 9950},
  {'name': 'Blueprint', 'itemCount': 1},
  {'name': 'Detonite Injector', 'itemCount': 5},
  {'name': 'Forma', 'itemCount': 1},
  {'name': 'Plastids